In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50


In [3]:
import cv2

def image_generator(directory, filenames, labels, batch_size=64, target_size=(48, 48), shuffle=True):
    num_samples = len(filenames)
    indices = np.arange(num_samples)
    if shuffle:
        np.random.shuffle(indices)

    while True:
        for start_idx in range(0, num_samples, batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            batch_images = []
            batch_labels = []
            for idx in batch_indices:
                filename = filenames[idx]
                label = labels[idx]
                img_path = os.path.join(directory, filename)
                
                # Resmi grayscale olarak okuma
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                # Resmi istenen boyuta yeniden boyutlandırma
                img = cv2.resize(img, target_size)
                # Resmi 0-255 aralığından 0-1 aralığına normalleştirme
                img_array = img / 255.0
                
                batch_images.append(img_array)
                batch_labels.append(label)

            yield np.array(batch_images), np.array(batch_labels)

In [4]:
dataset_dir = "D:/yapayzekafinal/resim2"  # Ana veri dizini
classes = os.listdir(dataset_dir)
num_classes = len(classes)
print(classes)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [5]:
def load_data():
    filenames = []
    labels = []
    for i, cls in enumerate(classes):
        class_dir = os.path.join(dataset_dir, cls)
        class_filenames = os.listdir(class_dir)
        class_filenames = [os.path.join(cls, filename) for filename in class_filenames]
        filenames.extend(class_filenames)
        labels.extend([i] * len(class_filenames))
    return filenames, labels

In [6]:
filenames, labels = load_data()
X_train_filenames, X_test_filenames, y_train, y_test = train_test_split(filenames, labels, test_size=0.2, random_state=42)


In [15]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [30]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [14]:
for layer in base_model.layers:
    layer.trainable = False

NameError: name 'base_model' is not defined

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [32]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [33]:
batch_size = 64
train_generator = image_generator(dataset_dir, X_train_filenames, y_train_categorical, batch_size=batch_size, target_size=(48, 48))
test_generator = image_generator(dataset_dir, X_test_filenames, y_test_categorical, batch_size=batch_size, target_size=(48, 48), shuffle=False)


In [34]:
steps_per_epoch = len(X_train_filenames) // batch_size
validation_steps = len(X_test_filenames) // batch_size

In [35]:
history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=34,
                    validation_data=test_generator,
                    validation_steps=validation_steps)

Epoch 1/34
358/358 ━━━━━━━━━━━━━━━━━━━━ 219s 603ms/step - accuracy: 0.2468 - loss: 1.8169 - val_accuracy: 0.3613 - val_loss: 1.6416
Epoch 2/34
358/358 ━━━━━━━━━━━━━━━━━━━━ 203s 566ms/step - accuracy: 0.3717 - loss: 1.5982 - val_accuracy: 0.4340 - val_loss: 1.4318
Epoch 3/34
358/358 ━━━━━━━━━━━━━━━━━━━━ 203s 566ms/step - accuracy: 0.4485 - loss: 1.4265 - val_accuracy: 0.4829 - val_loss: 1.3397
Epoch 4/34
358/358 ━━━━━━━━━━━━━━━━━━━━ 197s 551ms/step - accuracy: 0.4942 - loss: 1.3156 - val_accuracy: 0.5102 - val_loss: 1.2703
Epoch 5/34
358/358 ━━━━━━━━━━━━━━━━━━━━ 204s 571ms/step - accuracy: 0.5204 - loss: 1.2489 - val_accuracy: 0.5395 - val_loss: 1.2116
Epoch 6/34
213/358 ━━━━━━━━━━━━━━━━━━━━ 1:24 581ms/step - accuracy: 0.5461 - loss: 1.2047

KeyboardInterrupt: 

In [29]:
model.save("face.keras")